In [ ]:
# 1. Contexto del análisis

In [ ]:
# 2. El set de datps
'''La informacion recolectada se encuantra en un archivo CSV
(vic_elec_125256) con 54688 filas y 5 columnas'''

In [ ]:
# 3. Primera mirada al dataset
# Importamos librerias/modulos para EDA
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Leyendo el CSV
ruta = "./src/docs/vic_elec_125256.csv"
data = pd.read_csv(ruta)

In [ ]:
# Mostrar el dataset
print(data.shape)
data.head()

In [ ]:
# Veamos las varialbes categoricas y las numericas
data.info()

In [ ]:
# 4. Limpieza del dataset
'''Se realizara el proceso de limpieza teniendo encuanta las situaciones comunes:
1. Datos faltantes en algunas celdas.
2. Columnas irrelevantes (ques no corresponden al problema que queremos resolver)
3. Registros (filas) repetidos.
4. Valores extremos (outliers) en el caso de las variables numericas. Se deben
analizar en detalle pues no necesariamente la soulucion es eliminarlos.
5. Errore tipograficos en el caso de las variabe categoriccas.

Se supone que, al final de este proceso de limpieza deberiamos tener un set de datos
integro, listo para la fase de Análisis Exploratorio.
'''

In [ ]:
# 4.1 Datos faltantes
'''Aca comenzaremos viendo los datos que no esten comletos,
pues no todas las columnas tienen la misma cantidad de registros.
El numero totoal de registrs deberia ser 54,688.
'''

# Eliminar filas con celdas faltantes/vacias
data.dropna(inplace=True)
data.info()

In [ ]:
# 4.2 Columnas irrelevantes
'''Una columnas irrelenvante contiene:
1. No contienen informcion relevante para el problema que queremos resolver.
2. Una columnas categoria pero con un solo nivel.
3. Una columna numerica pero con un solo valor.
4. Columnas con informacion redundante.

Pero si se tiene dudas sobre una columnas puede ser relevante o no, lo mejor es dejarla,
y mas adelante en las sigueintes etapas, podremos darnos cuenta de si se preserva o no.
'''

# Conteo de los nievles en las diferentes columnas categoricas.
columnas_categoricas = ['Time', 'Demand', 'Temperature', 'Date', 'Holiday']

for columna in columnas_categoricas:
    print(f'Columna {columna}: {data[columna].nunique()} subniveles')

In [ ]:
'''Todas las columnas categoricas, tiene mas de un subnivel.
Lo cual no se elemina ninguna.
'''

# Veamos que ocurren con las columnas numericas
data.describe()

In [ ]:
'''Como se muestra, solo hay dos columnas que tienen desviaciones estandar (std)
diferentes de cero, lo que indica que no tiene un unico valor.
'''

In [ ]:
# 4.3 Filas repetidas
print(f'Volumen del dataset antes de eliminar filas repetidas: {data.shape}')
data.drop_duplicates(inplace=True)
print(f'Volumen del dataset despues de eliminar filas repetidas: {data.shape}')

In [ ]:
# 4.4 Outliers (valres extremos) en las variable numericas
'''No siempre se eliminan los Outliers porque dependiendo de la variable numerica
analizada, estos pueden contener informacion importante.

Crearemos graficas tipo "boxplot" de las columnas numericas:
'''

# Generarndo graficos individuales pues las varaible numericas
# estan en rangos diferentes

columnas_numericas = ['Demand', 'Temperature']

fig, ax = plt.subplots(nrows=7, ncols=1, figsize=(8, 30))
fig.subplots_adjust(hspace=0.5)

for i, col in enumerate(columnas_numericas):
    sns.boxplot(x=col, data=data, ax=ax[i])
    ax[i].set_title(col)

In [ ]:
'''Observaciones:
1. "Demand": hay valores con extremos a los 7000
2. "Temperature": hay valores con extremos a los 30 
'''

# Eliminar filas con "Demand" > 7000
print(f'Eliminando filas de Demand: \nVolumen del dataset antes de elimnar regristros de Demand: {
      data.shape}')
data = data[data['Demand'] >= 7000]
print(f'Volumen del dataset despues de elimnar regristros de Demand: {
      data.shape}\n')

# Eliminar filas con "Temperature" >30
print(f'Eliminando filas de Temperature: \nVolumen del dataset antes de elimnar regristros de Temperature: {
      data.shape}')
data = data[data['Temperature'] >= 25]
print(f'Volumen del dataset despues de elimnar regristros de Temperature: {
      data.shape}\n')

In [ ]:
# 4.5 Errores tipograficos en variable categoricas
'''En una variable categorica pueden aparecer subniveles como abreviatures
y palabras compretas, por ejemplo, "pag" y "paginas", lo cual para nosotros
son equivalentes, pero que a nuestro programa parerian diferentes.

Estos sub-niveles, deberian unificarse. Para este caso las caregorias, son unicas,
por lo cual no hay que unificarlos
'''

# Graficamos los subnivles de cada variable categorica
columnas_categoricas = ['Time', 'Demand', 'Temperature', 'Date', 'Holiday']

fig, ax = plt.subplots(nrows=5, ncols=1, figsize=(10, 30))
fig.subplots_adjust(hspace=1)

for i, col in enumerate(columnas_categoricas):
    sns.countplot(x=col, data=data, ax=ax[i])
    ax[i].set_title(col)
    ax[i].set_xticklabels(ax[i].get_xticklabels(), rotation=30)